# AI-Powered Fracture Detection — Colab Template
Team: Shashank, Suprita, Srujan

This notebook contains a step-by-step Colab-ready template for dataset download, preprocessing, training (transfer learning), evaluation, and Grad-CAM visualizations. Follow cells in order.

## 1) Notebook configuration & reproducibility

In [ ]:
# Project metadata and reproducibility settings
TEAM = ['Shashank', 'Suprita', 'Srujan']
DURATION_WEEKS = 6
DATA_DIR = '/content/data'  # adjust if mounting Drive
SEED = 42
import random, numpy as np, tensorflow as tf
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
print('Seeds set, TF version:', tf.__version__)

## 2) Environment & dependencies (Colab friendly)

In [ ]:
# Install extras if running in a fresh Colab session (comment out locally)
!pip install -q tensorflow albumentations opencv-python scikit-image scikit-learn 
import tensorflow as tf
print('GPU devices:', tf.config.list_physical_devices('GPU'))

## 3) Dataset download & folder layout (instructions)

In [ ]:
# Example: create canonical folders
from pathlib import Path
base = Path('/content/data')
for split in ['train','val','test']:
    for cls in ['fracture','normal']:
        (base/split/cls).mkdir(parents=True, exist_ok=True)
print(base)

## 4) Exploratory Data Analysis (samples & counts)

In [ ]:
import os
from collections import Counter
base = Path('/content/data')
for split in ['train','val','test']:
    counts = {cls: len(list((base/split/cls).glob('*'))) for cls in ['fracture','normal']}
    print(split, counts)

## 5) Preprocessing utilities (resize, normalize, to-RGB)

In [ ]:
from PIL import Image
import numpy as np
def load_image(path, target_size=(224,224)):
    img = Image.open(path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size)
    arr = np.array(img)/255.0
    return arr
# Example usage (replace with real file path)
# sample = load_image('/content/data/train/normal/example.jpg')
# print(sample.shape, sample.min(), sample.max())

## 6) Augmentation examples (Albumentations)

In [ ]:
import albumentations as A
import matplotlib.pyplot as plt
transform = A.Compose([
    A.Rotate(limit=15, p=0.7),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
])
# example application
# aug = transform(image=(sample*255).astype('uint8'))['image']
# plt.imshow(aug); plt.axis('off')

## 7) tf.data loader (efficient pipeline)

In [ ]:
import tensorflow as tf
def make_dataset(file_paths, labels, batch_size=16, image_size=(224,224), training=True):
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    def _load(path, label):
        img = tf.io.read_file(path)
        img = tf.io.decode_image(img, channels=3)
        img = tf.image.resize(img, image_size)
        img = img / 255.0
        return img, label
    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.shuffle(1024)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds
print('tf.data loader ready')

## 8) Baseline custom CNN (sanity check)

In [ ]:
from tensorflow.keras import layers, models
def small_cnn(input_shape=(224,224,3)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32,3,activation='relu')(inputs)
    x = layers.MaxPool2D(2)(x)
    x = layers.Conv2D(64,3,activation='relu')(x)
    x = layers.MaxPool2D(2)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    return model
m = small_cnn()
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
m.summary()

## 9) Transfer learning example (ResNet50)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
base = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base.trainable = False
x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
out = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs=base.input, outputs=out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## 10) Training loop & callbacks (example)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
ckpt = ModelCheckpoint('/content/best_model.h5', save_best_only=True, monitor='val_loss')
es = EarlyStopping(patience=5, restore_best_weights=True)
rlr = ReduceLROnPlateau(patience=3)
# history = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[ckpt, es, rlr])
print('Callbacks ready — run model.fit with your datasets')

## 11) Evaluation (metrics & confusion matrix)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
# preds = model.predict(test_ds)  # probabilities
# y_true = ... ; y_pred = (preds >= 0.5).astype(int)
# print(classification_report(y_true, y_pred))
print('Use sklearn.metrics for evaluation')

## 12) Grad-CAM (placeholder)

In [ ]:
# Placeholder for Grad-CAM utilities — implement grad_cam(model, img_tensor) to visualize attention
print('Implement Grad-CAM as a utility function and apply to misclassified samples')